# Demonstration of current state of non-nanosecond time in Pandas 
(as of 2.0.0.dev0+664.g86f182829d)  
We still need to use numpy to get to the previously out of bounds dates.   
This will be resolved once the `to_datetime` functionality is updated (estimated prior to 2.0)  

In [1]:
import numpy as np
import pandas as pd

### **[New Feature]** Dates far in the past

In [2]:
pd.Series(['1700-01-01'], dtype='datetime64[s]')

0   1700-01-01
dtype: datetime64[s]

### **[New Feature]** Dates far into the future represented as a Series

In [3]:
arr2 = np.array(["30456-01-02"], dtype="M8[s]")

# as series
pd.Series(arr2)

0   30456-01-02
dtype: datetime64[s]

### **[New Feature]** Dates far into the future represented as an Index

In [4]:
# or as a index
pd.Index(arr2)

DatetimeIndex(['30456-01-02'], dtype='datetime64[s]', freq=None)

### **[New Feature]** Long spanning time series

In [5]:
arr = np.arange(1000, dtype=np.int64).view("M8[100Y]").astype("M8[s]")
pd.Series(arr)

0       1970-01-01
1       2070-01-01
2       2170-01-01
3       2270-01-01
4       2370-01-01
          ...     
995   101470-01-01
996   101570-01-01
997   101670-01-01
998   101770-01-01
999   101870-01-01
Length: 1000, dtype: datetime64[s]

### **[New Feature]** Negative dates

In [6]:
arr3 = np.array(["-999999-01-02"], dtype="M8[s]")
pd.Series(arr3)

0   -999999-01-02
dtype: datetime64[s]

### By pandas 2.0.0, it will be possible to use negative dates directly in pandas:

In [7]:
try:
    pd.Series(["-999999-01-02"], dtype="datetime64[s]")
except:
    print("Not yet!")

Not yet!


### We can currently set a datum through `origin`
with limitations:
* Date must be numeric
* origin currently needs to be within the bounds,  1600-2300

In [12]:
pd.to_datetime(13, unit='D')

Timestamp('1970-01-14 00:00:00')

In [13]:
pd.to_datetime(13, unit='D', origin='1950-01-01')

Timestamp('1950-01-14 00:00:00')

In [17]:
pd.to_datetime(-13, unit='D', origin='1950-01-01')

Timestamp('1949-12-19 00:00:00')

### **[New Feature]** Series analysis using Index with dates far into the future

In [18]:
# dates far into the future
arr2 = np.array(["30456-01-02"], dtype="M8[s]")

# as series
df = pd.DataFrame(np.random.randint(0, 100, len(arr2)), index=pd.Index(arr2, name='date'))

# series.groupby(series.index.date).mean()
df.groupby(df.index.year).mean()

,0
date,
30456,40.0


In [19]:
# dates far into the future
start_year = 1300
stop_year = 13000

arr = np.arange(np.datetime64(f"{start_year}-01-01"), np.datetime64(f"{stop_year}-01-01"), np.timedelta64(100, "Y"), dtype='datetime64[Y]').astype("M8[s]")
series = pd.Series(np.random.randn(len(arr)), index=arr)

series.groupby(series.index.year).mean()

1300     0.981720
1400    -0.210187
1500    -0.613341
1600     0.808032
1700     0.147665
           ...   
12500    0.881354
12600    1.732501
12700    1.845402
12800   -0.770493
12900   -0.179177
Length: 117, dtype: float64